# Анализ  перевозок по файлам из http://brokenstone.ru/

In [1]:
# Импорт необходимых библиотек
import numpy as np
import pandas as pd

In [2]:
# Загрузка файла
file = "test_file.xlsx"
df = pd.read_excel(file)

In [3]:
# Список дорог
railways = (
    'ВСБ', 'ГОР', 'ДВС', 'ЖДЯ', 'ЗАБ', 'ЗСБ', 
    'КБШ', 'КЛГ', 'КРС', 'МСК', 'ОКТ', 'ПРВ', 
    'СВР', 'СЕВ', 'СКВ', 'ЮВС', 'ЮУР')

In [4]:
# Исключения
place_of_calculation = ["Перевозки ОАО РЖД"]

# Фильтры
type_of_transportation = ["Внутр. перевозки"]
cargo_group = ["Минерально-строит."]
cargo_OKVED = ["Разработка карьеров"]
type_of_van = ["Полувагоны"]
departure_railways = ["СКВ","ОКТ","ЮУР"]

**Предобработка данных**

In [5]:
# Применение фильтров и исключений
df = df.loc[df['Вид перевозки'].isin(type_of_transportation)]
df = df.loc[df['Группа груза'].isin(cargo_group)]
df = df.loc[df['Груз ОКВЭД'].isin(cargo_OKVED)]
df = df.loc[df['Род вагона'].isin(type_of_van)]
df = df.loc[~df['Признак места расчета'].isin(place_of_calculation)]

In [6]:
# Преобразование наименований организаций
def clearing_organizational_form(dataframe):
    dataframe.replace(regex=True, inplace=True, to_replace=r'АО "ДВ ТРАНСПОРТНАЯ ГРУППА"   АО "ДВТГ"', value=r'ДВТГ')
    dataframe.replace(regex=True, inplace=True, to_replace=r'^И.П.', value=r'ИП')
    dataframe.replace(regex=True, inplace=True, to_replace=r'  ', value=r' ')
    dataframe.replace(regex=True, inplace=True, to_replace=r'(?i)Общество с ограниченной ответственностью', value=r'ООО')
    dataframe.replace(regex=True, inplace=True, to_replace=r'(?i)^Акционерное общество', value=r'АО')
    dataframe.replace(regex=True, inplace=True, to_replace=r'(?i)Государственное унитарное предприятие', value=r'ГУП')
    dataframe.replace(regex=True, inplace=True, to_replace=r'(?i)Открытое акционерное общество', value=r'ОАО')
    dataframe.replace(regex=True, inplace=True, to_replace=r'(?i)Закрытое акционерное общество', value=r'ЗАО')
    dataframe.replace(regex=True, inplace=True, to_replace=r'(?i)Публичное акционерное общество', value=r'ПАО')
    dataframe.replace(regex=True, inplace=True, 
                      to_replace=r'(?i)(^OOO |^АО |^ООО |^ОАО |^ЗАО |^ОАО|^ООО|^ПАО | ООО$| АО$| ЗАО$| ОАО$| ООО$| ПАО$|[^\w\d\s+-.])', 
                      value=r'')
    dataframe.replace(regex=True, inplace=True, to_replace=r'(?i)ФЕДЕРАЛЬНАЯ ГРУЗОВАЯ КОМПАНИЯ', value=r'ФГК')
    dataframe.replace(regex=True, inplace=True, to_replace=r'(?i)Новая перевозочная компания', value=r'НПК')
    dataframe.replace(regex=True, inplace=True, to_replace=r'(?i)Холдинговая компания', value=r'ХК')
    dataframe.replace(regex=True, inplace=True, to_replace=r'ЭЙ-СИ-РЕЙЛ', value=r'Эй-Си-Рейл')
    dataframe.replace(regex=True, inplace=True, to_replace=r'Кузбасстрансцемент', value=r'КузбассТрансЦемент')
    dataframe.replace(regex=True, inplace=True, to_replace=r'Новотранс ХК', value=r'ХК Новотранс')
    dataframe.replace(regex=True, inplace=True, to_replace=r'ХК НОВОТРАНС', value=r'ХК Новотранс')
    dataframe.replace(regex=True, inplace=True, to_replace=r'^\s', value=r'')

clearing_organizational_form(df['Собственник вагона по ЕГРПО'])
clearing_organizational_form(df['Арендатор вагона по ГВЦ'])
clearing_organizational_form(df['Арендатор по внутреннему справочнику'])
clearing_organizational_form(df['Плательщик жд тарифа'])
clearing_organizational_form(df['Грузоотправитель наим по ЕГРПО'])
clearing_organizational_form(df['Грузополучатель наим по ЕГРПО'])

In [7]:
# Владелец вагона (арендатор или собственник)
df['Арендатор вагона по ГВЦ'].apply(lambda x: x.lower())
df['Владелец вагона'] = np.where(df['Арендатор вагона по ГВЦ'] == 'вне аренды', df['Собственник вагона по ЕГРПО'], df['Арендатор вагона по ГВЦ'])

In [8]:
# Столбцы, которые необходимо оставить
df_new = df[[
    'Номер документа', 'Дата оформления', 'Наименование груза', 'Дорога отправления',
    'Субъект РФ отправления', 'Станция отправления РФ', 'Грузоотправитель наим по ЕГРПО',
    'Дорога назначения', 'Субъект РФ назначения', 'Станция назначения РФ', 'Грузополучатель наим по ЕГРПО',
    'Плательщик жд тарифа', 'Ваг*Км (тариф Расстояние)', 'Объем перевозок тонн', 'Провозная плата', 
    'Владелец вагона', 'Номер вагона', 'Модель вагона']].reset_index(drop=True)
df_new = df_new.loc[df_new['Дорога отправления'].isin(departure_railways)]

**Анализ по собственникам вагонов**

In [9]:
# Сводная таблица по собственникам вагонов
vag_pivot_table = df.pivot_table(
    columns=['Дорога отправления','Дорога назначения'], 
    values='Признак вагонных перевозок', 
    aggfunc=np.sum, 
    index='Собственник вагона по ЕГРПО', 
    fill_value=0)

In [10]:
# Топ собственников вагонов по дороге отправления
def vag_top(departure_railway, count: int = 30):
    df_railway = df[df['Дорога отправления'] == departure_railway]
    vag_railway_pivot_table = df_railway.pivot_table(
        columns=['Дорога назначения'], 
        index='Владелец вагона', 
        values='Объем перевозок тонн', 
        aggfunc=len, 
        fill_value=0, 
        margins=True)
    vag_railway_top = vag_railway_pivot_table.sort_values(by='All', ascending=False).head(count+1)
    vag_railway_top = vag_railway_top.drop(index=['All'])
    return vag_railway_top

In [11]:
skv_vag_top = vag_top(departure_railway='СКВ', count=30)
okt_vag_top = vag_top(departure_railway='ОКТ', count=30)
yur_vag_top = vag_top(departure_railway='ЮУР', count=30)

**Анализ по грузополучателям**

In [12]:
# Топ грузополучателей по дороге назначения
def consignees_top(destinations_railway, count: int = 30):
    df_railway = df[df['Дорога назначения'] == destinations_railway]
    consignee_railway_pivot_table = df_railway.pivot_table(
        columns=['Дорога отправления'], 
        index='Грузополучатель наим по ЕГРПО', 
        values='Объем перевозок тонн', 
        aggfunc=np.sum, 
        fill_value=0, 
        margins=True)
    consignee_railway_top = consignee_railway_pivot_table.sort_values(by='All', ascending=False).head(count+1)
    consignee_railway_top = consignee_railway_top.drop(index=['All'])
    consignee_railway_top = consignee_railway_top.loc[:, (consignee_railway_top != 0).any(axis=0)]
    return consignee_railway_top

In [13]:
msk_consignees_top = consignees_top(destinations_railway='МСК', count=30)
gor_consignees_top = consignees_top(destinations_railway='ГОР', count=30)

**Запись в файл**

In [22]:
# Генерация имени конечного файла
date_min = df['Дата оформления'].min().date().strftime("%d.%m.%Y")
date_max = df['Дата оформления'].max().date().strftime("%d.%m.%Y")
name_file_output = f'Перевозки + анализ {date_min}-{date_max}'

In [23]:
# Запись в файл
writer = pd.ExcelWriter(f'{name_file_output}.xlsx')
df_new.to_excel(writer, 'rzd', index=False)
vag_pivot_table.to_excel(writer, 'vag')
skv_vag_top.to_excel(writer, 'skv_vag')
okt_vag_top.to_excel(writer, 'okt_vag')
msk_consignees_top.to_excel(writer, 'to_msk')
gor_consignees_top.to_excel(writer, 'to_gor')
writer.save()